In [1]:
import os
from pathlib import Path

from torch.autograd import Variable
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import albumentations as albu
import numpy as np
import cv2
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt

import config as cfg

/home/shung/miniconda3/envs/STAS/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
OUTPUT_PATH = "./OpenVINO"
BASE_MODEL_NAME = OUTPUT_PATH + "/Deeplabv3P_resnet50_batch16"

IMAGE_WIDTH = 1600
IMAGE_HEIGHT = 800
DEVICE='cuda'

In [3]:
if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

# Paths where PyTorch, ONNX and OpenVINO IR models will be stored.
model_path = Path(BASE_MODEL_NAME).with_suffix(".pth")
onnx_path = model_path.with_suffix(".onnx")
ir_path = model_path.with_suffix(".xml")

In [4]:
dummy_input = torch.randn(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH).to(DEVICE)
model = torch.load('./Models/Deeplabv3P_resnet50_batch16.pth')
model.eval()
torch.onnx.export(model.module, dummy_input, onnx_path, verbose=True, )

Exported graph: graph(%input.1 : Float(1, 3, 800, 1600, strides=[3840000, 1280000, 1600, 1], requires_grad=0, device=cuda:0),
      %decoder.aspp.0.convs.1.0.0.weight : Float(2048, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %decoder.aspp.0.convs.2.0.0.weight : Float(2048, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %decoder.aspp.0.convs.3.0.0.weight : Float(2048, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %decoder.aspp.1.0.weight : Float(256, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %decoder.block2.0.0.weight : Float(304, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %segmentation_head.0.weight : Float(1, 256, 1, 1, strides=[256, 1, 1, 1], requires_grad=1, device=cuda:0),
      %segmentation_head.0.bias : Float(1, strides=[1], requires_grad=1, device=cuda:0),
      %onnx::Conv_617 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cuda:

In [5]:
mo_cmd = f"""mo
            --input_model "{onnx_path}"
            --input_shape "[1, 3, {IMAGE_HEIGHT}, {IMAGE_WIDTH}]"
            --data_type FP16
            --output_dir "{OUTPUT_PATH}"
            """
mo_cmd = " ".join(mo_cmd.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO:")
display(Markdown(f"`{mo_cmd}`"))

Model Optimizer command to convert the ONNX model to OpenVINO:


`mo --input_model "OpenVINO/Deeplabv3P_resnet50_batch16.onnx" --input_shape "[1, 3, 800, 1600]" --data_type FP16 --output_dir "./OpenVINO"`

In [6]:
if not ir_path.exists():
    print("Exporting ONNX model to IR... This may take a few minutes.")
    mo_result = %sx $mo_cmd
    print("\n".join(mo_result))
    print("DONE")
else:
    print(f"IR model {ir_path} already exists.")

IR model OpenVINO/Deeplabv3P_resnet50_batch16.xml already exists.
